In [10]:
import sys
sys.path.append('..')

import numpy as np
from pytenet.hartree_fock_mps import generate_single_state
from pytenet.operation import add_mps, operator_average
from pytenet.hamiltonian_thc import eval_func, generate_thc_mpos_by_layer_qn, get_t, get_h1_spin, get_g_spin
from pytenet.global_krylov_method import generate_krylov_space_in_disk, get_W, get_S, generate_re_ortho_space_with_coeff, coeff_canonical_orthogonalization,  generate_linear_combination_mps
from pytenet.global_krylov_method import solve_ritz, generate_reduced_H_non_ortho, remain_only_tridiagonal_elements, coeff_gram_schmidt, generate_Hamiltonian_with_occupation_number, solve_ritz_two_vec
import numpy as np
from scipy import sparse
import copy
import h5py
from numpy.linalg import norm
#np.set_printoptions(precision=4,suppress=True)
import scipy.io
import matplotlib.pyplot as plt
import pickle
import pytenet as ptn
from pytenet.operation_thc import get_num_op_mpo


Load and initialize datas: 

no is number of spatial orbitals

L is number of spinor orbitals, L = 2*no

t_spin is one-body integral in Chemist's notation (considering spins)

g_spin is two-body integral in Chemist's notation (considering spins)

X_mo and Z_mo are THC tensors, X_mo_up/down are X_mo considering spins

r_THC is THC rank

In [11]:
#load integrals
with h5py.File("/work_fast/ge49cag/code_datas/NH3/integral.hdf5", "r") as f:
#with h5py.File("/work_fast/ge49cag/pytenet_yu/water/eri_water.hdf5", "r") as f:
    eri = f["eri"][()]
    hkin = f["hkin"][()]
    hnuc = f["hnuc"][()]

#print(np.linalg.norm(eri))
#print(eri.shape)

no = eri.shape[0]
MV = eri.reshape(no*no,no*no)

u = np.load("/work_fast/ge49cag/code_datas/NH3/x.npy")
#u = np.load("/work_fast/ge49cag/pytenet_yu/water/x.npy")
X_mo = u.transpose(1,0)
g_thc, Z_mo = eval_func(u,eri,hkin,hnuc,)
h1 = hnuc+hkin
nmo = X_mo.shape[1]
L = 2*X_mo.shape[1]
g_thc = g_thc.reshape(nmo, nmo, nmo, nmo)
r_thc = X_mo.shape[0]

8
(8, 36)
(36, 36)
rl errV: 6.421105521729929e-13
abs errV: 4.286591936668418e-12
errt: 1.0574710399920436e-13
errh: 1.8088404989173088e-14
errht: 9.006422291220954e-14


In [12]:
# H_correct = get_molecular_Hamiltonian_as_sparse_matrix(get_h1_spin(h1), get_g_spin(eri))
H_correct = scipy.io.mmread('/work_fast/ge49cag/code_datas/H_correct_NH3.mtx').tocsr()
# H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
# sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

In [ ]:
H_correct_10e = generate_Hamiltonian_with_occupation_number(H_correct.real, 10)
e ,v = sparse.linalg.eigsh(H_correct_10e, which='SA', k = 15)

/home/ge49cag/.local/lib/python3.10/site-packages/scipy/sparse/_index.py:143: SparseEfficiencyWarning: Changing the sparsity structure of a csr_matrix is expensive. lil_matrix is more efficient.
  self._set_arrayXarray(i, j, x)


In [ ]:
e_ground = e[0]
e_1st_ex = e[1]

We can calculate elements in reduced Hamiltonian using conventional MPO.

Since we only need to store ONE block during contraction, memory needed is only $\mathcal{O}(L^2 M^2)$.

Create conventional mpo for molecular Hamiltonian:

In [ ]:
# h1_spin = get_h1_spin(h1)
# g_spin = get_g_spin(eri)
g_phy =  eri.transpose(0, 2, 1, 3)
#mpo_ref = ptn.hamiltonian.molecular_hamiltonian_mpo(h1_spin, g_spin_phy)
mpo_ref = ptn.hamiltonian.spin_molecular_hamiltonian_mpo(h1, g_phy)
print(mpo_ref.bond_dims)

[1, 16, 62, 100, 154, 100, 62, 16, 1]


Generate THC-MPO by layers, using THC tensors. 
t_spin is used to create MPO for kinetic term.
It returns a list of H_mu_nu, each H_mu_nu is also a list, which contains four smaller MPOs with bond dims 2.
The final element of this list is MPO for kinetic term.

In [ ]:
#generate thc_mpo
t = get_t(h1, eri)
H_mu_nu_list_spin_layer = generate_thc_mpos_by_layer_qn(X_mo, Z_mo, L, t)

print(type(H_mu_nu_list_spin_layer))
print(type(H_mu_nu_list_spin_layer[0]))
print(type(H_mu_nu_list_spin_layer[0][0]))
print((H_mu_nu_list_spin_layer[0][0].bond_dims))

<class 'list'>
<class 'list'>
<class 'pytenet.mpo.MPO'>
[1, 2, 2, 2, 2, 2, 2, 2, 1]


For ground state finding, we use Hatree fock state |11111111110000> as initial state.

For 1st excited state, please use single-excited Hatree-Fock state as initial state, or even superposition of several single-excited Hatree-Fock states as initial state.

In [ ]:
#temp1 = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 0, 0, 0, 0])
#initial = generate_single_state(16, [1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0])

HFS1 = generate_single_state(8, [3, 3, 3, 3, 3, 0, 0, 0])
initial = copy.deepcopy(HFS1)
#HFS2 = generate_single_state(8, [3, 3, 3, 3, 1, 2, 0, 0])
#initial = copy.deepcopy(HFS2)
#initial = add_mps(HFS1, HFS2)
#initial.orthonormalize('left')
#initial.orthonormalize('right')
#hartree_state = add_mps(add_mps(add_mps(temp1, temp2), temp3),temp4)
# hartree_state = add_mps(temp1, temp2)
# hartree_state.orthonormalize('left')
# hartree_state.orthonormalize('right')

generate Krylov space:

In [ ]:
N_Krylov_1 = 30
psi_original_1 = copy.deepcopy(initial)
#psi_original_1 = copy.deepcopy(initial)
max_bond_Krylov_1 = 120
trunc_tol = 1e-12
foldername_1 = f"/work_fast/ge49cag/code_datas/NH3_ground"
generate_krylov_space_in_disk(N_Krylov_1, H_mu_nu_list_spin_layer, psi_original_1, max_bond_Krylov_1, trunc_tol, r_thc, foldername_1)

KeyboardInterrupt: 

In [ ]:
H_reduced_non_rotho_1 = generate_reduced_H_non_ortho(N_Krylov_1, foldername_1, mpo_ref)
coeff_1 = coeff_gram_schmidt(N_Krylov_1, foldername_1)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_1 = np.einsum('ik, kl, jl -> ij', coeff_1.conj(), H_reduced_non_rotho_1, coeff_1)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:213: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)


/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:112: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [ ]:
e_ritz_1, v_ritz_1 = solve_ritz(foldername_1, H_reduced_1, N_Krylov_1, coeff_1, max_bond_Krylov_1, e_ground, mpo_ref)

(0.8734029443100866+0j)


(0.4039462471330779+0j)
(0.39774169014697236+0j)
(0.39698543563865485+0j)


In [ ]:
# e_krylov_1, v_krylov_1 = np.linalg.eigh(H_reduced_1)
# shift = e_krylov_1[0]
# Q_1, R_1 = np.linalg.qr(H_reduced_1 - shift*np.identity(H_reduced_1.shape[0]))

# coeff_shifted_1 = np.einsum('ai, ab -> ib', Q_1, coeff_1)
# #H_shifted_1 = np.einsum('ik, kl, jl -> ij', coeff_shifted_1.conj(), H_reduced_non_rotho_1, coeff_shifted_1)
# #H_shifted_1 = coeff_shifted_1 @H_reduced_non_rotho_1 @coeff_shifted_1.T 

Restart:

In [ ]:
N_Krylov_2 = 30
psi_original_2 = copy.deepcopy(v_ritz_1)
max_bond_Krylov_2 = 120
trunc_tol = 0
foldername_2= f"/work_fast/ge49cag/code_datas/NH3_ground_restart1"
generate_krylov_space_in_disk(N_Krylov_2, H_mu_nu_list_spin_layer, psi_original_2, max_bond_Krylov_2, trunc_tol, r_thc, foldername_2)


[1, 4, 16, 64, 120, 64, 16, 4, 1]
2
[1, 4, 16, 64, 120, 64, 16, 4, 1]
3
[1, 4, 16, 64, 120, 64, 16, 4, 1]
4
[1, 4, 16, 64, 120, 64, 16, 4, 1]
5
[1, 4, 16, 63, 120, 64, 16, 4, 1]
6
[1, 4, 16, 64, 120, 64, 16, 4, 1]
7
[1, 4, 16, 64, 120, 64, 16, 4, 1]
8
[1, 4, 16, 64, 120, 64, 16, 4, 1]
9
[1, 4, 16, 64, 120, 64, 16, 4, 1]
10
[1, 4, 16, 64, 120, 64, 16, 4, 1]
11
[1, 4, 16, 64, 120, 64, 16, 4, 1]
12
[1, 4, 16, 64, 120, 64, 16, 4, 1]
13
[1, 4, 16, 64, 120, 64, 16, 4, 1]
14
[1, 4, 16, 64, 120, 64, 16, 4, 1]
15
[1, 4, 16, 64, 120, 64, 16, 4, 1]
16
[1, 4, 16, 64, 120, 64, 16, 4, 1]
17
[1, 4, 16, 64, 120, 64, 16, 4, 1]
18
[1, 4, 16, 64, 120, 64, 16, 4, 1]
19
[1, 4, 16, 64, 120, 64, 16, 4, 1]


In [ ]:
H_reduced_non_rotho_2 = generate_reduced_H_non_ortho(N_Krylov_2, foldername_2, mpo_ref)
coeff_2 = coeff_gram_schmidt(N_Krylov_2, foldername_2)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_2 = np.einsum('ik, kl, jl -> ij', coeff_2.conj(), H_reduced_non_rotho_2, coeff_2)
#H_reduced_2 = remain_only_tridiagonal_elements(H_reduced_2)

In [ ]:
e_ritz_2, v_ritz_2 = solve_ritz(foldername_2, H_reduced_2, N_Krylov_2, coeff_2, max_bond_Krylov_2, e_ground, mpo_ref)

(0.370153375611153+0j)


(0.31848834468840437+0j)
(0.31706865299264564+0j)
(0.31458036490968766+0j)


Restart:

In [ ]:
N_Krylov_3 = 20
psi_original_3 = copy.deepcopy(v_ritz_2)
max_bond_Krylov_3 = 120
trunc_tol = 0
foldername_3 = f"/work_fast/ge49cag/code_datas/NH3_ground_restart2"
generate_krylov_space_in_disk(N_Krylov_3, H_mu_nu_list_spin_layer, psi_original_3, max_bond_Krylov_3, trunc_tol, r_thc, foldername_3)

[1, 4, 16, 63, 120, 64, 16, 4, 1]
2
[1, 4, 16, 64, 120, 64, 16, 4, 1]
3
[1, 4, 16, 64, 120, 64, 16, 4, 1]
4
[1, 4, 16, 64, 120, 64, 16, 4, 1]
5
[1, 4, 16, 64, 120, 64, 16, 4, 1]
6
[1, 4, 16, 64, 120, 64, 16, 4, 1]
7
[1, 4, 16, 63, 120, 64, 16, 4, 1]
8
[1, 4, 16, 64, 120, 64, 16, 4, 1]
9
[1, 4, 16, 64, 120, 64, 16, 4, 1]
10
[1, 4, 16, 64, 120, 64, 16, 4, 1]
11


KeyboardInterrupt: 

In [ ]:
H_reduced_non_rotho_3 = generate_reduced_H_non_ortho(N_Krylov_3, foldername_3, mpo_ref)
coeff_3 = coeff_gram_schmidt(N_Krylov_3, foldername_3)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_3 = np.einsum('ik, kl, jl -> ij', coeff_3.conj(), H_reduced_non_rotho_3, coeff_3)

In [ ]:
e_ritz_3, v_ritz_3 = solve_ritz(foldername_3, H_reduced_3, N_Krylov_3, coeff_3, max_bond_Krylov_3, e_ground, mpo_ref)

(1.7996599612744006+0j)
(1.1216790599749515+0j)
(1.1580559356631284+0j)
(1.828487211792961+0j)
(2.5460892326990603+0j)


(2.5562386401052777+0j)


Restart:

In [ ]:
N_Krylov_4 = 20
#psi_original_4 = copy.deepcopy(v_ritz_3)
max_bond_Krylov_4 = 120
trunc_tol = 0
foldername_4 = f"/work_fast/ge49cag/code_datas/NH3_ground_restart3"
#generate_krylov_space_in_disk(N_Krylov_4, H_mu_nu_list_spin_layer, psi_original_4, max_bond_Krylov_4, trunc_tol, r_thc, foldername_4)


In [ ]:
H_reduced_non_rotho_4 = generate_reduced_H_non_ortho(N_Krylov_4, foldername_4, mpo_ref)
coeff_4 = coeff_gram_schmidt(N_Krylov_4, foldername_4)
#H_reduced: elements calculated by post-orthogonalized Krylov vectos
H_reduced_4 = np.einsum('ik, kl, jl -> ij', coeff_4.conj(), H_reduced_non_rotho_4, coeff_4)

/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:213: ComplexWarning: Casting complex values to real discards the imaginary part
  H_reduced[i, j] = operator_inner_product(temp1, H_mpo, temp2)
/work_fast/ge49cag/pytenet_thc_spin_cons/thc_experiments/../pytenet/global_krylov_method.py:112: ComplexWarning: Casting complex values to real discards the imaginary part
  W[i,j] = vdot(temp1, temp2)


In [ ]:
e_ritz_4, v_ritz_4 = solve_ritz(foldername_4, H_reduced_4, N_Krylov_4, coeff_4, max_bond_Krylov_4, e_ground, mpo_ref)

(3.054239385846813e-07+0j)
(3.026812152029379e-07+0j)
(3.0097088199454447e-07+0j)
(2.9967691261845175e-07+0j)
(2.98798696007907e-07+0j)
(2.985627531870705e-07+0j)
(2.97496058010438e-07+0j)
(2.9258103495521937e-07+0j)


In [ ]:
# filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_ground_state.pkl"
# with open(filename, 'wb') as file:
#     pickle.dump(v_ritz_4, file)

# filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_ground_state.pkl"
# with open(filename, 'rb') as file:
#     NH3_ground = pickle.load(file)

# operator_average(NH3_ground, mpo_ref)

(-67.469235802238+0j)

In [ ]:
# e_4, v_4 = np.linalg.eigh(H_reduced_4)
# NH3_1st_excited =  generate_linear_combination_mps(N_Krylov_4, v_4[:,1], max_bond_Krylov_4, foldername_4)
# print(operator_average(NH3_1st_excited, mpo_ref) - e_1st_ex)

# filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_1st_excited.pkl"
# with open(filename, 'wb') as file:
#     pickle.dump(NH3_1st_excited, file)
    
# filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_1st_excited.pkl"
# with open(filename, 'rb') as file:
#     NH3_1st_excited = pickle.load(file)

# print(operator_average(NH3_1st_excited, mpo_ref) - e_1st_ex)

(0.007366645143619621+0j)


In [ ]:
# to_remove = copy.deepcopy(NH3_ground)
# to_remove.A[0] = -ptn.operation.vdot(to_remove, copy.deepcopy(NH3_1st_excited))* to_remove.A[0]
# #compress the bond dims back 
# NH3_1st_excited =  ptn.operation_thc.add_mps_and_compress(copy.deepcopy(NH3_1st_excited), to_remove, 0, 200)
# NH3_1st_excited.orthonormalize('right')
# print(operator_average(NH3_1st_excited, mpo_ref) - e_1st_ex)

(0.009297126598710292+0j)


In [ ]:
# filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_1st_excited.pkl"
# with open(filename, 'wb') as file:
#     pickle.dump(NH3_1st_excited, file)

# filename = f"/work_fast/ge49cag/code_datas" + f"/NH3_1st_excited.pkl"
# with open(filename, 'rb') as file:
#     NH3_1st_excited = pickle.load(file)

# print(operator_average(NH3_1st_excited, mpo_ref) - e_1st_ex)

(0.009297126598710292+0j)


In [ ]:
# -ptn.operation.vdot(NH3_1st_excited, NH3_ground)

(6.572509802572381e-17-0j)